In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pickle
from pprint import pprint
from tqdm import tqdm
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
import optuna

from utils import load_data_from_directory, split_into_K_folds, get_metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")


In [2]:
train_images_filenames = pickle.load(open('../MIT_split/train_images_filenames.dat','rb'))
test_images_filenames = pickle.load(open('../MIT_split/test_images_filenames.dat','rb'))
train_images_filenames = ['..' + n[15:] for n in train_images_filenames]
test_images_filenames  = ['..' + n[15:] for n in test_images_filenames]
train_labels = pickle.load(open('../MIT_split/train_labels.dat','rb')) 
test_labels = pickle.load(open('../MIT_split/test_labels.dat','rb'))

In [3]:
# Since we want to do k-fold cross validation, we need to merge the training and test data and then re-split it into k folds

all_data = train_images_filenames + test_images_filenames
all_labels = train_labels + test_labels

k = 5  # number of folds
# split data into 5 folds using custom function
folds = split_into_K_folds(all_data, all_labels, k)

# Now each fold in the 'folds' list contains the training and test data for that fold in the format: (train_data, train_labels, test_data, test_labels)
for fold_index, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    print(f"Fold {fold_index + 1}:")
    print(f"Number of samples in train set: {len(train_data)}")
    print(f"Number of samples in test set: {len(test_data)}")
    print(f"Number of labels in train set: {len(train_labels)}")
    print(f"Number of labels in test set: {len(test_labels)}")
    print()

Fold 1:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 2:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 3:
Number of samples in train set: 2150
Number of samples in test set: 538
Number of labels in train set: 2150
Number of labels in test set: 538

Fold 4:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537

Fold 5:
Number of samples in train set: 2151
Number of samples in test set: 537
Number of labels in train set: 2151
Number of labels in test set: 537



In [4]:
# Define Dense SIFT and Dense KAZE descriptors
def dense_sift(image, step_size=8, kp_scale=16):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dense_sift = cv2.SIFT_create()
    keypoints = [cv2.KeyPoint(x, y, kp_scale) for y in range(0, image.shape[0], step_size) for x in range(0, image.shape[1], step_size)]
    _, descriptor = dense_sift.compute(image, keypoints)
    return descriptor

# generate descriptors for all images and store in dictionary to access them later in each fold without having to recompute them
def generate_descriptors(image_paths, descriptor_type, **kwargs):
    descriptors = {}
    for image_path in tqdm(image_paths):
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if descriptor_type == 'sift':
            sift = cv2.SIFT_create()
            _, descriptor = sift.detectAndCompute(gray_image, None)
        elif descriptor_type == 'kaze':
            kaze = cv2.KAZE_create(threshold=kwargs['threshold'])
            _, descriptor = kaze.detectAndCompute(gray_image, None)
        elif descriptor_type == 'dense_sift':
            descriptor = dense_sift(image, step_size=kwargs['step_size'], kp_scale=kwargs['kp_scale'])
        else:
            raise Exception('Unknown descriptor type')
        descriptors[image_path] = descriptor
    return descriptors

descriptors_dictionary = generate_descriptors(all_data, 'dense_sift', step_size=16, kp_scale=16, threshold=0.0001)


  0%|          | 0/2688 [00:00<?, ?it/s]

100%|██████████| 2688/2688 [00:40<00:00, 66.48it/s]


In [5]:
# create a dictionary linking a histogram of visual words to the corresponding images images:
def create_visual_words(image_paths, descriptors_dictionary, codebook):
    visual_words = {}
    for image_path in image_paths:
        descriptor = descriptors_dictionary[image_path]
        labels = codebook.predict(descriptor)
        visual_words[image_path] = np.bincount(labels, minlength=codebook.n_clusters)
    return np.array(list(visual_words.values()))


In [6]:
codebooks = {'fold_0': {}, 'fold_1': {}, 'fold_2': {}, 'fold_3': {}, 'fold_4': {}}
codebook_sizes = [32, 64, 128, 256, 512, 1024] #, 2048, 4096, 8192]

for i, fold in enumerate(folds):
    train_data, train_labels, test_data, test_labels = fold
    train_descriptors = []

    for file in train_data:
        train_descriptors.append(descriptors_dictionary[file])
    train_D = np.vstack(list(train_descriptors)) 
    for k in tqdm(codebook_sizes, desc=f'Creating Codebooks for fold {i}'):

        codebook = MiniBatchKMeans(n_clusters=k, 
                           verbose=False, 
                           batch_size=2150,
                           compute_labels=False,
                           reassignment_ratio=0.01,
                           random_state=42)
        
        codebook.fit(train_D)

        codebooks[f'fold_{i}'][k] = codebook


Creating Codebooks for fold 4: 100%|██████████| 6/6 [00:29<00:00,  4.84s/it]


In [8]:
gamma_vals = [0.05, 0.1, 0.15, 0.2, 0.25]
c_vals = [0.01, 0.1, 1, 10, 100]
#pca_comp = [8, 16, 32, 64]

def objective(trial):

    n_clusters = trial.suggest_categorical('n_clusters', codebook_sizes)
    #dimensionality_reduction = trial.suggest_categorical('dimensionality_reduction', ['pca', 'lda', 'none'])

    svm_kernel = trial.suggest_categorical('svm_kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
    svm_C = trial.suggest_categorical('svm_C', c_vals)
    svm_gamma = trial.suggest_categorical('svm_gamma', gamma_vals)

    
    #if dimensionality_reduction == 'pca':
    #    pca_n_components = trial.suggest_int('pca_n_components', 1, n_clusters, step=32)
    #elif dimensionality_reduction == 'lda':
    #    lda_n_components = trial.suggest_int('lda_n_components', 1, 7)

    metrics = []
    # Run k-fold cross validation per hyperparameter combination
    for i, fold in enumerate(folds, 0):

        train_data, train_labels, test_data, test_labels = fold
        
        codebook = codebooks[f'fold_{i}'][n_clusters]
        # create visual words for training data
        train_visual_words = create_visual_words(train_data, descriptors_dictionary, codebook)
        # create visual words for test data
        test_visual_words = create_visual_words(test_data, descriptors_dictionary, codebook)
        
        
        # fit SVM classifier
        print(f'Fold {i}: fitting svm with C={svm_C}, kernel={svm_kernel}, gamma={svm_gamma}, n_clusters={n_clusters}')
        svm = SVC(C=svm_C, kernel=svm_kernel, gamma=svm_gamma)
        svm.fit(train_visual_words, train_labels)
        # predict labels for test data
        pred_labels = svm.predict(test_visual_words)
    
        # get accuracy metric for current run
        metrics.append(get_metrics(test_labels, pred_labels)[0])

    return sum(metrics)/len(metrics)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

study.best_params

[I 2024-01-04 23:52:35,420] A new study created in memory with name: no-name-56f7c4ff-c71f-405d-8cd2-562ee200e547


Fold 0: fitting svm with C=100, kernel=rbf, gamma=0.1, n_clusters=64
Fold 1: fitting svm with C=100, kernel=rbf, gamma=0.1, n_clusters=64
Fold 2: fitting svm with C=100, kernel=rbf, gamma=0.1, n_clusters=64
Fold 3: fitting svm with C=100, kernel=rbf, gamma=0.1, n_clusters=64
Fold 4: fitting svm with C=100, kernel=rbf, gamma=0.1, n_clusters=64


[I 2024-01-04 23:52:45,525] Trial 0 finished with value: 0.15326161450437165 and parameters: {'n_clusters': 64, 'svm_kernel': 'rbf', 'svm_C': 100, 'svm_gamma': 0.1}. Best is trial 0 with value: 0.15326161450437165.


Fold 0: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=128
Fold 1: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=128
Fold 2: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=128
Fold 3: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=128
Fold 4: fitting svm with C=0.1, kernel=poly, gamma=0.1, n_clusters=128


[I 2024-01-04 23:52:53,722] Trial 1 finished with value: 0.8084179629360415 and parameters: {'n_clusters': 128, 'svm_kernel': 'poly', 'svm_C': 0.1, 'svm_gamma': 0.1}. Best is trial 1 with value: 0.8084179629360415.


...

Fold 4: fitting svm with C=0.1, kernel=poly, gamma=0.2, n_clusters=64
Fold 0: fitting svm with C=0.01, kernel=linear, gamma=0.15, n_clusters=1024
Fold 1: fitting svm with C=0.01, kernel=linear, gamma=0.15, n_clusters=1024
Fold 2: fitting svm with C=0.01, kernel=linear, gamma=0.15, n_clusters=1024
Fold 3: fitting svm with C=0.01, kernel=linear, gamma=0.15, n_clusters=1024
Fold 4: fitting svm with C=0.01, kernel=linear, gamma=0.15, n_clusters=1024


[I 2024-01-05 00:31:41,762] Trial 98 finished with value: 0.839292364990689 and parameters: {'n_clusters': 1024, 'svm_kernel': 'linear', 'svm_C': 0.01, 'svm_gamma': 0.15}. Best is trial 19 with value: 0.839292364990689.


Fold 0: fitting svm with C=0.01, kernel=linear, gamma=0.2, n_clusters=1024
Fold 1: fitting svm with C=0.01, kernel=linear, gamma=0.2, n_clusters=1024
Fold 2: fitting svm with C=0.01, kernel=linear, gamma=0.2, n_clusters=1024
Fold 3: fitting svm with C=0.01, kernel=linear, gamma=0.2, n_clusters=1024
Fold 4: fitting svm with C=0.01, kernel=linear, gamma=0.2, n_clusters=1024


[I 2024-01-05 00:32:10,367] Trial 99 finished with value: 0.839292364990689 and parameters: {'n_clusters': 1024, 'svm_kernel': 'linear', 'svm_C': 0.01, 'svm_gamma': 0.2}. Best is trial 19 with value: 0.839292364990689.


{'n_clusters': 1024, 'svm_kernel': 'linear', 'svm_C': 0.01, 'svm_gamma': 0.2}